In [89]:
import random 

sample = open('tracks.txt', 'w')

print("track_id x_start y_start z_start x_end y_end z_end dE", file=sample)

for i in range(int(1e2)):
    for j in range(int(random.random()*1000)):
        print(i, random.random(), random.random(), random.random(), 
              2*random.random(), 2*random.random(), 2*random.random(), 
              abs(random.random())*10, file=sample)
    
sample.close()

In [90]:
import pandas as pd
import numpy as np
import torch

In [131]:
tracks = pd.read_csv('tracks.txt', delim_whitespace=True)

tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) + 
                       pow(tracks['y_end']-tracks['y_start'], 2) +
                       pow(tracks['z_end']-tracks['z_start'], 2))

tracks['x'] = (tracks['x_end']-tracks['x_start'])/2
tracks['y'] = (tracks['y_end']-tracks['y_start'])/2
tracks['z'] = (tracks['z_end']-tracks['z_start'])/2

tracks['dE'] = np.abs(tracks['dE'])
tracks['dEdx'] = tracks['dE']/tracks['dx']
tracks['dEdx'][tracks['dEdx'] < 1] = 1
tracks = tracks.drop(columns=['x_start','y_start','z_start','x_end','y_end','z_end'])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ttracks = torch.tensor(tracks.values).to(device)

ix = tracks.columns.get_loc("x")
idE = tracks.columns.get_loc("dE")
idx = tracks.columns.get_loc("dx")
idEdx = tracks.columns.get_loc("dEdx")


In [138]:

class IonizationElectrons(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        self._MeVToElectrons = 4.237e+04
        self._alpha   = 0.847
        self._beta    = 0.2061
        self._vdrift  = 0.153812 # cm/us
        self._lifetime = 6000.  # ms

        super(IonizationElectrons, self).__init__()

        
    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        add_column = torch.nn.ZeroPad2d((0, 1, 0, 0))
        x = add_column(x)

        recomb = torch.log(self._alpha + self._beta * x[:,idEdx]) / (self._beta * x[:,idEdx])
        lifetime = torch.exp( -1. * x[:,ix] / self._vdrift / self._lifetime)
        x[:,-1] = self._MeVToElectrons * x[:,idE] * recomb * lifetime
        
        return x

In [143]:
ionizationElectrons = IonizationElectrons()
ttracks = ionizationElectrons(ttracks)

tensor([0.9997, 1.0002, 0.9992,  ..., 0.9993, 1.0004, 1.0000],
       dtype=torch.float64)


In [146]:
head = ttracks[:10]

In [216]:
head.size()
torch.ceil(head[0][-1])
tracks = head
tracks

tensor([[ 0.0000e+00,  8.7146e+00,  1.8030e+00,  2.9352e-01,  4.8586e-01,
          7.0034e-01,  4.8334e+00,  2.2658e+05,  2.2658e+05],
        [ 0.0000e+00,  5.9056e+00,  1.0336e+00, -1.4539e-01,  1.5261e-01,
          4.7189e-01,  5.7134e+00,  1.4990e+05,  1.4990e+05],
        [ 0.0000e+00,  5.1408e+00,  1.8252e+00,  7.0177e-01,  3.7837e-01,
          4.4403e-01,  2.8166e+00,  1.3345e+05,  1.3345e+05],
        [ 0.0000e+00,  3.4608e+00,  3.7379e-01,  1.6124e-01,  9.4322e-02,
          5.9868e-03,  9.2587e+00,  7.7868e+04,  7.7868e+04],
        [ 0.0000e+00,  6.1849e+00,  1.4931e+00,  6.3315e-01, -1.7205e-01,
          3.5615e-01,  4.1424e+00,  1.6290e+05,  1.6290e+05],
        [ 0.0000e+00,  1.8484e+00,  1.3534e+00,  3.0020e-01, -3.6456e-01,
          4.8465e-01,  1.3657e+00,  3.3619e+04,  3.3619e+04],
        [ 0.0000e+00,  3.8517e+00,  2.0974e+00,  2.6614e-01,  3.9730e-01,
          9.3330e-01,  1.8365e+00,  8.7651e+04,  8.7651e+04],
        [ 0.0000e+00,  1.9746e-01,  1.5656e+00, 

In [207]:
clusters = torch.zeros((tracks[0][-1]/60).int(), 1)

for t in tracks[1:]:
    cluster = torch.zeros((t[-1]/60).int(), 1)
    clusters = torch.cat((clusters, cluster))

torch.Size([18458, 1])